In [1]:
pip install streamlit pyngrok python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [2]:
from threading import Thread
from pyngrok import ngrok
from google.colab import userdata
import os

In [3]:
ngrok_token = userdata.get("NGROK_AUTH_KEY")

ngrok.set_auth_token(ngrok_token)

In [4]:
def run_streamlit():
  os.system("streamlit run /content/app.py --server.port 8501")

In [21]:
%%writefile app.py

import PIL
from PIL import Image
import streamlit as st
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import plotly.graph_objects as go
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall

import google.generativeai as genai
from google.colab import userdata
from PIL import Image
import os

from google.colab import userdata
from dotenv import load_dotenv
load_dotenv()

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

output_dir = "saliency_maps"
os.makedirs(output_dir, exist_ok=True)

def generate_explanation(img_path, model_prediction, confidence):
    prompt = f"""You are an experienced brain surgeon. You are tasked with explaining a saliency map of a brain tumor MRI scan.
    The saliency map was generated by a deep learning model that was trained to classify brain tumors
    as either glioma, meningioma, pituitary, or no tumor.

    The saliency map highlights the regions of the image that the machine learning model is focusing on to make the prediction.

    The deep learning model predicted the image to be of class '{model_prediction}' with a confidence of {confidence * 100}%.

    In your response:
    - Explain what regions of the brain the model is focusing on, based on the saliency map. Refer to the regions highlighted
    in light cyan, those are the regions where the model is focusing on.
    - Explain possible reasons why the model made the prediction it did.
    - Don't mention anything like 'The saliency map highlights the regions the model is focusing on, which are in light cyan', we want actual detalied eplanations of what the ML model was focusing on and how the area indicates that there is a case of glioma, or no brain tumor, or etc.
    - Keep your explanation to 4 sentences max.
    - After your explanation give a couple of next steps that should be taken by the patient. Remember you are a brain surgeon so you should give them advice and statistics on the severity of their tumor and what they can do next. Do NOT give them false hope or no hope, but be honest.

    Let's think step by step about this. Verify step by step.
    """

    img = PIL.Image.open(img_path)

    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    response = model.generate_content([prompt, img])

    return response.text



def generate_saliency_map(model, img_array, class_index, img_size):
  with tf.GradientTape() as tape:
    img_tensor = tf.convert_to_tensor(img_array)
    tape.watch(img_tensor)
    predictions = model(img_tensor)
    target_class = predictions[:, class_index]

  gradients = tape.gradient(target_class, img_tensor)
  gradients = tf.math.abs(gradients)
  gradients = tf.reduce_max(gradients, axis=-1)
  gradients = gradients.numpy().squeeze()

  gradients = cv2.resize(gradients, img_size)

  center = (gradients.shape[0] // 2, gradients.shape[1] // 2)
  radius = min(center[0], center[1])
  y, x, = np.ogrid[:gradients.shape[0], :gradients.shape[1]]
  mask = (x - center[0])**2 + (y- center[1])**2  <= radius**2

  gradients = gradients * mask

  brain_gradients = gradients[mask]
  if brain_gradients.max() > brain_gradients.min():
    brain_gradients = ((brain_gradients - brain_gradients.min()) / ((brain_gradients.max()) - brain_gradients.min()))
  gradients[mask] = brain_gradients

  threshold = np.percentile(gradients[mask], 80)
  gradients[gradients < threshold] = 0

  gradients = cv2.GaussianBlur(gradients, (11, 11), 0)

  heatmap = cv2.applyColorMap(np.uint8(255 * gradients), cv2.COLORMAP_JET)
  heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

  heatmap = cv2.resize(heatmap, img_size)

  original_img = image.img_to_array(img)
  superimposed_img = heatmap * 0.7 + original_img * 0.3
  superimposed_img = superimposed_img.astype(np.uint8)

  img_path = os.path.join(output_dir, uploaded_file.name)
  with open(img_path, "wb") as f:
    f.write(uploaded_file.getbuffer())

  saliency_map_path = f"saliency_maps/{uploaded_file.name}"

  cv2.imwrite(saliency_map_path, cv2.cvtColor(superimposed_img, cv2.COLOR_RGB2BGR))

  return superimposed_img

def load_xception_model(model_path):
  img_shape = (299, 299, 3)
  base_model = tf.keras.applications.Xception(
    include_top = False,
    weights = 'imagenet',
    input_shape=img_shape,
    pooling='max'
  )

  model = Sequential([
    base_model,
    Flatten(),
    Dropout(rate=0.3),
    Dense(128, activation='relu'),
    Dropout(rate=0.25),
    Dense(4, activation='softmax')
  ])

  model.build((None,) + img_shape)

  model.compile(
    Adamax(learning_rate=0.001),
    loss='catergorical_crossentropy',
    metrics=['accuracy', Precision(), Recall()]
  )
  model.load_weights(model_path)

  return model

st.title("Brain Tumor Classification")

st.write("Upload an image of a brain MRI scan to begin classification process")

uploaded_file = st.file_uploader("Choose an image", type=("jpg", "jpeg", "png"))

if uploaded_file is not None:
  selected_model = st.radio(
    "Select Model",
    ("Transfer Learning - Xception", "Custom CNN")
  )

  if selected_model == "Transfer Learning - Xception":
    model = load_xception_model('/content/xception_model.weights.h5')
    img_size = (299, 299)
  else:
    model = load_model('/content/cnn_model.h5')
    img_size = (224, 224)

  labels = ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary']
  img = image.load_img(uploaded_file, target_size=img_size)
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array /= 255.0

  prediction = model.predict(img_array)

  class_index = np.argmax(prediction[0])
  result = labels[class_index]

  st.write(f"Predicted Class: {result}")
  st.write("Predictions:")
  for label, prob in zip(labels, prediction[0]):
    st.write(f"{label}: {prob:.4f}")

  saliency_map = generate_saliency_map(model, img_array, class_index, img_size)

  col1, col2 = st.columns(2)
  with col1:
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
  with col2:
    st.image(saliency_map, caption="Saliency Map", use_column_width=True)

  st.write("## Classification Results")

  result_container = st.container()
  result_container = st.container()
  some_confidence = (prediction[0][class_index] * 100)
  if some_confidence == 100:
    font_color = '#00FF00' #Green
  if some_confidence > 80 and some_confidence < 100:
    font_color = '#0000FF' #Blue
  if some_confidence > 75 and some_confidence < 80:
    font_color = '#FFFF00' #Yellow
  if some_confidence < 75:
    font_color = '#FF0000' #Red

  if result == 'No Tumor':
    text_font_color = '#FFFFFF' #White
  else:
    text_font_color = '#FF0000' #Red

  result_container.markdown(
    f"""
    <div style="background-color: #000000; color: #ffffff; padding: 30px; border-radius: 15px;">
        <div style="display: flex; justify-content: space-between; align-items: center;">
            <div style="flex: 1; text-align: center;">
                <h3 style="color: #ffffff; margin-bottom: 10px; font-size: 20px;">Prediction</h3>
                <p style="font-size: 36px; font-weight: 800; color: {text_font_color}; margin: 0;">
                    {result}
                </p>
            </div>
            <div style="width: 2px; height: 80px; background-color: #ffffff; margin: 0 20px;"></div>
            <div style="flex: 1; text-align: center;">
                <h3 style="color: #ffffff; margin-bottom: 10px; font-size: 20px;">Confidence</h3>
                <p style="font-size: 36px; font-weight: 800; color: {font_color}; margin: 0;">
                    {prediction[0][class_index] * 100:.4}%
                </p>
            </div>
        </div>
    </div>
    """,
    unsafe_allow_html=True
  )

  # Prepare data for Plotly chart
  probabilities = prediction[0]
  sorted_indices = np.argsort(probabilities)[::-1]
  sorted_labels = [labels[i] for i in sorted_indices]
  sorted_probabilities = probabilities[sorted_indices]

  # Create a Plotly bar chart
  fig = go.Figure(go.Bar(
    x=sorted_probabilities,
    y=sorted_labels,
    orientation='h',
    marker_color=['red' if label == result else 'blue' for label in sorted_labels]
  ))

  # Customize the chart layout
  fig.update_layout(
    title='Probabilities for each class',
    xaxis_title='Probability',
    yaxis_title='Class',
    height=400,
    width=600,
    yaxis=dict(autorange='reversed')
  )

  # Add value labels to the bars
  for i, prob in enumerate(sorted_probabilities):
    fig.add_annotation(
        x=prob,
        y=i,
        text=f'{prob:.4f}',
        showarrow=False,
        xanchor='left',
        xshift=5
    )

  # Display the Plotly chart
  st.plotly_chart(fig)


  saliency_map_path = f'saliency_maps/{uploaded_file.name}'
  explanation = generate_explanation(saliency_map_path, result, prediction[0][class_index])

  st.write("### Explanation:")
  st.write(explanation)

Overwriting app.py


In [6]:
active_tunnels = ngrok.get_tunnels()
for tunnel in active_tunnels:
    print(f"Killing existing tunnel: {tunnel.public_url}")
    ngrok.disconnect(tunnel.public_url)

In [7]:
thread = Thread(target=run_streamlit)
thread.start()

In [8]:
public_url = ngrok.connect(addr='8501', proto='http', bind_tls=True)

print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://dc59-34-83-227-74.ngrok-free.app" -> "http://localhost:8501"


Overwriting .env
